# Imports

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from sklearn.preprocessing import MinMaxScaler

# Helper Functions 

## Helper Functions for cleaning data from Ballchasing.com

In [2]:
def get_totals(df):
    # Select only the columns that contain the total counts for each statistic over the course of the season.
    total = df[[
    'team name', 'player name', 'games', 'wins', 'score', 'goals', 'assists', 'saves', 'shots', 'shots conceded', 'goals conceded',
    'goals conceded while last defender', 'amount collected', 'amount collected big pads', 'amount collected small pads',  
    'count collected big pads', 'count collected small pads','amount stolen', 'amount stolen big pads', 'amount stolen small pads',
    'count stolen big pads', 'count stolen small pads', '0 boost time', '100 boost time', 'amount used while supersonic',
    'amount overfill total','amount overfill stolen', 'total distance', 'time slow speed', 'time boost speed', 'time supersonic speed',
    'time on ground', 'time low in air', 'time high in air', 'time powerslide','count powerslide','time most back', 'time most forward',
    'time in front of ball','time behind ball', 'time defensive half', 'time offensive half', 'time defensive third', 'time neutral third',
    'time offensive third','demos inflicted', 'demos taken'
    ]]
    # Lowercase all playernames for consistency.
    total['player name'] = total["player name"].str.lower() 
    # Since we only have a dataframe of total statistics, we can group by player to account for BallChasing.com accidently mismanaging 
    # team and player relationships. This is acting as a merge between rows where playernames match but the teams names didnt. 
    cleaned_data = total.groupby(['player name']).sum().reset_index()
    return cleaned_data

In [3]:
def average_player_statistics(cleaned_data):
    # In order to round the data to the nearest thousandth, we need to remove all columns without int datatypes. I divide the total
    # statistics for every player by the total number of (recorded) games that they have played and uploaded replays. 
    per_game_stats = cleaned_data.iloc[:,2:].div(cleaned_data.games, axis=0).round(4)
    # This is theseries that contains all columns without int datatypes, in other words all of the players. 
    players = cleaned_data.iloc[:,:2]
    # Then we merge these back together for the final dataframe.
    final_df = pd.concat([players, per_game_stats], axis=1)
    # return df so we can 
    return final_df

In [4]:
def remove_player_statistics(df, lst):
    # Provide as input a list of player names to be drop from this dataframe. 
    return df[~df['player name'].isin(players_to_drop)]

## Helper Functions for generating player standings 

Current Season = TRL Winter 2021 Tournament

Win Rate

Offensive Rating
- score per game + shots per game + assists per game + shooting percentage + time most forward per game + time offensive half per game + time neurtal third per game + time offensive third per game

Defensive Rating
- score per game + saves per game + goals conceded per game + goals conceded while last defender per game + time most back per game + time defensive half per game + time neurtal third per game + time defensive third per game

Aggression
- shots per game + average speed per game + amount stolen per game + amount stolen big pads per game + amount stolen small pads per game + time offensive half per game + demos inflicted per game + demos taken per game + time powerslide per game + avg powerslide time per game + count powerslide per game

Speed
- bpm per game + avg boost amount per game + time supersonic per game +total distance per game + avg speed per game + average amount used while supersonic per game + amount collected per game + amount collected big pads per game + amount collected small pads per game + 0 boost time per game + 100 boost time per game + amount used while supersonic per game + time slow speed per game + time boost speed per game + time supersonic speed per game

In [5]:
def offensive_stats(df):
    # Select all columns with relevant data 
    Offensive = df[[
        'player name', 'score', 'shots', 'assists',
        'time most forward','time offensive half',
        'time neutral third','time offensive third'
        ]]
    # Create a new column with the calculated offensive rating value based on original column data calculations
    Offensive['Offensive Rating'] = .05*Offensive['score'] + .2*Offensive['shots'] \
                                + .05*Offensive['assists'] + .05*Offensive['time most forward'] + .05*Offensive['time neutral third'] \
                                + .1*Offensive['time offensive third'] 
    # The we use the MinMaxScaler from SKLearn to normalize all of the values and to generate ranks. 
    scaler = MinMaxScaler()
    Offensive['Offensive Rating']  = scaler.fit_transform(Offensive[['Offensive Rating']])
    # Return the DataFrame containing [player name, offensive rating] 
    # Columns in sorted order with respect to Offensive rating  
    return Offensive.sort_values('Offensive Rating',ascending=False)[['player name', 'Offensive Rating']]

In [6]:
def defensive_stats(df):
    # Select all columns with relevant data 
    Defensive = df[[
        'player name', 'score', 'saves','goals conceded','goals conceded while last defender',
        'time most back','time defensive half','time neutral third','time defensive third'
        ]]
    # Create a new column with the calculated defensive rating value based on original column data calculations
    Defensive['Defensive Rating'] = .05* Defensive['score']  + .2*Defensive['saves'] + .05*Defensive['goals conceded'] +\
                                    .1*Defensive['goals conceded while last defender']+ .1*Defensive['time most back'] +\
                                    .15*Defensive['time defensive half'] + .1 * Defensive['time neutral third'] +\
                                    .05 * Defensive['time defensive third']
    # The we use the MinMaxScaler from SKLearn to normalize all of the values and to generate ranks. 
    scaler = MinMaxScaler()
    Defensive['Defensive Rating']  = scaler.fit_transform(Defensive[['Defensive Rating']])
    # Return the DataFrame containing [player name, defensive rating] 
    # Columns in sorted order with respect to Offensive rating  
    return Defensive.sort_values('Defensive Rating',ascending=False)[['player name', 'Defensive Rating']]

In [7]:
def aggression_stats(df):
    # Select all columns with relevant data 
    Aggression = df[[
        'player name', 'shots', 'amount stolen', 'amount stolen big pads', 'amount stolen small pads',
        'time offensive half','demos inflicted','demos taken','time powerslide','count powerslide'
        ]]
    # Create a new column with the calculated defensive rating value based on original column data calculations
    Aggression['Aggression Rating'] = .2*Aggression['shots'] +.025*Aggression['amount stolen']+ .05*Aggression['amount stolen big pads']+\
                                        .025*Aggression['amount stolen small pads'] + .075*Aggression['time offensive half'] +\
                                        .2*Aggression['demos inflicted'] + .025*Aggression['demos taken'] +\
                                        .05*Aggression['time powerslide'] + .15*Aggression['count powerslide']
    # The we use the MinMaxScaler from SKLearn to normalize all of the values and to generate ranks.
    scaler = MinMaxScaler()
    Aggression['Aggression Rating']  = scaler.fit_transform(Aggression[['Aggression Rating']])
    # Return the DataFrame containing [player name, aggression rating] 
    # Columns in sorted order with respect to aggression rating  
    return Aggression.sort_values('Aggression Rating',ascending=False)[['player name', 'Aggression Rating']]

In [16]:
def speed_stats(df):
    # Select all columns with relevant data 
    Speed = df[[
        'player name','total distance','amount collected','amount collected big pads','amount collected small pads',
        '0 boost time','100 boost time','amount used while supersonic','time slow speed','time boost speed','time supersonic speed'
        ]]
    # Create a new column with the calculated defensive rating value based on original column data calculations
    Speed['Speed Rating'] = + 4*Speed['total distance']\
                        + (-3*Speed['amount used while supersonic']) + 1*Speed['amount collected']\
                        + 1*Speed['amount collected big pads'] + 2*Speed['amount collected small pads'] + (-3*Speed['0 boost time'])\
                        + 3*Speed['100 boost time'] + -1*Speed['time slow speed'] + 1*Speed['time boost speed']\
                        + 3*Speed['time supersonic speed']
    # The we use the MinMaxScaler from SKLearn to normalize all of the values and to generate ranks.
    scaler = MinMaxScaler()
    # Return the DataFrame containing [player name, speed rating] 
    # Columns in sorted order with respect to speed rating  
    Speed['Speed Rating'] = scaler.fit_transform(Speed[['Speed Rating']])
    return Speed.sort_values('Speed Rating',ascending=False)[['player name', 'Speed Rating']]

## Helper Functions to Generate Final Overall Standings DataFrame

In [24]:
def generate_overall_standings(df, offensive_stats, defensive_stats, aggression_stats, speed_stats):
    # Take name and win rate from cleaned dataframe
    final_df = df[['player name', 'wins']]
    # Add the offensive player ratings to final df by merging on playername.
    final_df = final_df.merge(offensive_stats, how='left')
    # Add the defensive player ratings to final df by merging on playername.
    final_df = final_df.merge(defensive_stats, how='left')
    # Add the aggression player ratings to final df by merging on playername.
    final_df = final_df.merge(aggression_stats, how='left')
    # Add the speed player ratings to final df by merging on playername.
    final_df = final_df.merge(speed_stats, how='left')

    # Formatting by reordering columns in a readable format. 
    overalls = final_df[['player name','wins','Offensive Rating', 'Defensive Rating', 'Aggression Rating', 'Speed Rating']]

    # Generate overall standings based on all generated stats
    overalls['Total Overall'] = .3*overalls['Offensive Rating'] + .2*overalls['Defensive Rating'] +\
                                .35*overalls['Aggression Rating'] + .15*overalls['Speed Rating']
    # Scaler for normalization                             
    scaler = MinMaxScaler()
    # Normalize
    overalls['Total Overall'] = scaler.fit_transform(overalls[['Total Overall']])
    # Sort values by total overall score
    total_overalls = overalls.sort_values('Total Overall',ascending=False)

    # Formatting for easier human readability
    total_overalls.reset_index(drop=True).round(3)
    total_overalls['Offensive Rating'] = total_overalls['Offensive Rating'] *100 
    total_overalls['Defensive Rating'] = total_overalls['Defensive Rating'] *100 
    total_overalls['Aggression Rating'] = total_overalls['Aggression Rating'] *100 
    total_overalls['Speed Rating'] = total_overalls['Speed Rating'] *100 
    total_overalls['Total Overall'] = total_overalls['Total Overall'] *100 

    # Display the final dataframe for people to inspect
    results = total_overalls[[
        'player name', 'wins', 'Total Overall',
        'Offensive Rating', 'Defensive Rating', 'Aggression Rating', 'Speed Rating']]\
            .round(2).reset_index(drop=True)\
                .rename(columns={
                    "player name": "participant",
                    'wins': 'win rate',
                    "Total Overall": "Overall",
                    'Offensive Rating': 'Offense',
                    'Defensive Rating': 'Defense',
                    'Aggression Rating':'Aggression',
                    'Speed Rating': 'Speed'})

    return results


# Load in Fall 2020 player data

In [9]:
fall_2020 = 'data\TRL_F20_playerdata_final.csv'
df = pd.read_csv(fall_2020)
fall_2020_totals = get_totals(df)
fall_2020_averages = average_player_statistics(fall_2020_totals)

# Load in Winter Player Data

In [10]:
winter_2021 = 'data\TRL_W21_playerdata.csv'
winter_2021 = pd.read_csv(winter_2021)
# winter_2021.head()
winter_2021_totals = get_totals(winter_2021)
winter_2021_averages = average_player_statistics(winter_2021_totals)

In [13]:
# If there are any bots or players that shouldnt be in this dataframe, we can remove them with the next helper function. 
players_to_drop = ['squishy', 'tag cramification', 'yegs', 'goofy']
TRL_W21_WK1 = remove_player_statistics(winter_2021_averages, players_to_drop)

In [14]:
TRL_W21_WK1.head()

,player name,games,wins,score,goals,assists,saves,shots,shots conceded,goals conceded,goals conceded while last defender,amount collected,amount collected big pads,amount collected small pads,count collected big pads,count collected small pads,amount stolen,amount stolen big pads,amount stolen small pads,count stolen big pads,count stolen small pads,0 boost time,100 boost time,amount used while supersonic,amount overfill total,amount overfill stolen,total distance,time slow speed,time boost speed,time supersonic speed,time on ground,time low in air,time high in air,time powerslide,count powerslide,time most back,time most forward,time in front of ball,time behind ball,time defensive half,time offensive half,time defensive third,time neutral third,time offensive third,demos inflicted,demos taken
0,adamamango,16,0.5625,308.8750,0.5000,0.6250,1.3125,2.1875,7.0000,2.9375,1.2500,1958.6250,1354.4375,604.1875,16.5000,52.2500,418.8125,259.8125,159.0000,3.1875,14.1250,37.1238,33.0513,155.7500,291.1250,52.0000,504460.8125,183.6219,151.8619,33.9175,217.1369,143.4581,8.8025,8.2150,50.0625,136.6188,107.6500,100.6025,268.7962,241.7719,127.6269,175.4225,120.1619,73.8150,0.3750,1.0000
1,alex,10,0.5000,423.0000,1.0000,0.5000,1.6000,3.9000,7.4000,1.6000,0.5000,2112.8000,1557.8000,555.0000,20.2000,47.8000,433.1000,275.2000,157.9000,3.6000,13.8000,27.1490,38.3350,224.7000,479.0000,87.1000,505873.3000,148.6090,145.2860,46.5290,192.4760,134.3310,13.6130,5.4500,52.8000,98.4900,131.4100,77.8540,262.5670,214.9510,125.4730,157.5580,109.3640,73.4990,1.0000,1.4000
2,brictone,9,0.1111,313.2222,0.5556,0.2222,1.5556,2.0000,11.1111,4.5556,1.6667,2208.1111,1406.8889,801.2222,17.0000,71.4444,452.2222,269.3333,182.8889,3.2222,15.3333,47.7511,23.5144,241.1111,209.2222,24.5556,519643.2222,191.8667,151.4822,34.1578,235.7833,131.0656,10.6556,8.5278,67.1111,149.7667,169.8111,128.6300,248.8778,264.5378,112.9578,200.4967,115.7611,61.2478,0.5556,1.1111
3,casillas361,10,0.5000,136.5000,0.3000,0.3000,0.6000,0.8000,7.4000,1.6000,0.5000,1708.0000,1145.2000,562.8000,13.2000,47.4000,226.9000,101.6000,125.3000,1.2000,10.8000,56.5620,26.6830,351.2000,161.3000,18.8000,478884.0000,169.6070,143.4820,29.7830,240.7610,95.8530,6.2580,10.3330,91.6000,117.7400,95.0500,82.3240,260.5480,242.4530,100.4170,185.9450,101.2400,55.6880,0.2000,0.6000
4,chyaboi,15,0.4667,352.4667,0.8000,0.2667,1.2667,2.6000,6.8667,1.8667,0.6667,2409.4667,1723.6000,685.8667,20.2667,57.0000,443.8000,256.2667,187.5333,2.9333,14.7333,52.3807,34.3160,258.4000,263.7333,26.8667,511015.8667,159.9353,145.7980,47.2707,199.0840,140.2393,13.6813,7.8713,73.0000,116.7400,109.1867,93.3007,259.7060,235.5227,117.4827,174.9527,110.1773,67.8780,0.8000,1.0000


In [20]:
offensive_df = offensive_stats(TRL_W21_WK1)
defensive_df = defensive_stats(TRL_W21_WK1)
aggression_df = aggression_stats(TRL_W21_WK1)
speed_df = speed_stats(TRL_W21_WK1)

In [25]:
standings = generate_overall_standings(TRL_W21_WK1, offensive_df, defensive_df, aggression_df, speed_df)

In [26]:
standings

,participant,win rate,Overall,Offense,Defense,Aggression,Speed
0,invincible,0.55,100.00,90.44,28.42,100.00,79.40
1,nsdlakers4,0.40,83.61,81.53,71.32,40.29,100.00
2,ix mini,0.73,80.52,100.00,46.22,45.74,68.83
3,greensleeves,0.53,78.58,93.45,48.23,55.88,46.19
4,invincibleblaze,0.60,77.98,78.51,15.21,80.50,59.76
5,luisito,0.44,71.05,76.49,57.74,52.48,38.88
6,rj5588,0.11,70.62,85.51,100.00,1.90,80.46
7,terminator,0.73,69.29,85.76,42.36,48.40,41.84
8,shaunch,0.50,69.28,73.16,21.79,61.51,63.84
9,muffled,0.45,63.90,79.57,37.13,41.79,50.56
